# Task 3: A/B Hypothesis Testing


Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


Load Dataset

In [ ]:
df = pd.read_csv("../data/insurance.txt", delimiter="|")
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']
df['LossRatio'] = df['LossRatio'].replace([np.inf, -np.inf], np.nan)
df['ClaimOccurred'] = df['TotalClaims'] > 0
df.head()


📊 Hypotheses Tested
1. Province vs Risk (ANOVA)

In [ ]:
province_groups = df[['Province', 'LossRatio']].dropna().groupby('Province')['LossRatio'].apply(list)
f_stat, p_val = stats.f_oneway(*province_groups)


2. Zip Code vs Risk

In [ ]:
top_zipcodes = df['PostalCode'].value_counts().head(2).index
group1 = df[df['PostalCode'] == top_zipcodes[0]]['LossRatio'].dropna()
group2 = df[df['PostalCode'] == top_zipcodes[1]]['LossRatio'].dropna()
t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)


3. Margin Difference by Zip Code

In [ ]:
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
group1 = df[df['PostalCode'] == top_zipcodes[0]]['Margin'].dropna()
group2 = df[df['PostalCode'] == top_zipcodes[1]]['Margin'].dropna()
t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)


4. Gender vs Risk

In [ ]:
group_m = df[df['Gender'] == 'Male']['LossRatio'].dropna()
group_f = df[df['Gender'] == 'Female']['LossRatio'].dropna()
t_stat, p_val = stats.ttest_ind(group_m, group_f, equal_var=False)
